In [22]:
import argparse
import json
from datetime import datetime, timedelta, date
from pathlib import Path
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from dateutil.relativedelta import *
#from pycoingecko import CoinGeckoAPI
from tqdm import tqdm



def get_token_data(token, api_key, end_date, start_date, put_date_info=True):
    params = {
        "format": "csv",
        "key": api_key,
        "from": start_date,
        "to": end_date,
    }
    url = f"https://api.covalenthq.com/v1/pricing/historical/usd/{token}/"
    data_history_response = requests.get(url=url, params=params)
    data_history_response = data_history_response.text.split('\n')
    if len(data_history_response) < 365:
        print(f'Exclude {token} token, because of unavailability of its data!')
        return None

    data_history_response = data_history_response[:-1] # remove the last item (it's empty)
    headers = data_history_response[0]
    data_history_response = data_history_response[1:] # remove the header
    if put_date_info:
        date_info = [[x.split(',')[0].strip('"'), x.split(',')[1].strip('"'), None] for x in data_history_response] # remove redundant double quote
    else:
        date_info = [[x.split(',')[1].strip('"'), None] for x in data_history_response] # remove redundant double quote

    return date_info



def get_historical_data(token, api_key, end_date, window):
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d").date()
    start_date_dt = end_date_dt - timedelta(days=window)
    start_date = start_date_dt.strftime("%Y-%m-%d")
    print(f"start date: {start_date}, end date: {end_date}")
    data_history_response = get_token_data(token, api_key, end_date, start_date)
    new_headers = ["Date", "Price", "Token", "Symbol"]
    df = pd.DataFrame(data_history_response, columns=new_headers)
    print(f"The length of the received data is: {len(data_history_response)}")  # Print the url
    print(f"Just show the head of dataframe:\n{df.head()}")

    return df



def get_tokens_history(tokens, api_key, end_date, window):
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d").date()
    start_date_dt = end_date_dt - timedelta(days=window)
    start_date = start_date_dt.strftime("%Y-%m-%d")
    print(f"start date: {start_date}, end date: {end_date}, window: {window}")
    excluded_tokens = []
    headers = ["date"]
    data = []
    for idx, token in enumerate(tqdm(tokens)):
        data_history = get_token_data(token, api_key, end_date, start_date, True if idx == 0 else False)
        if data_history is None:
            excluded_tokens.append(token)
            continue

        if idx == 0:
            data = data_history
        else:
            data = [a + b for a, b in zip(data, data_history)]

        headers += [token, token + "_priceChange"]
        

    df = pd.DataFrame(data, columns=headers)
    df = df.sort_values(by='date', ascending = True)
    tokens = [token for token in tokens if token not in excluded_tokens]
    
    for i in range(df.index.size):
        if not i:
            continue

        for token in tokens:
            # Calculate price chnage relative to 1
            df[token + "_priceChange"].iloc[i] = str(((float(df[token].iloc[i]) - float(df[token].iloc[i-1])) / float(df[token].iloc[i-1])) + 1)
            
            
    df.drop(0, inplace=True)

    return df




def main(args):
    tokens = args.tokens
    chain = args.chain
    api_key = args.api_key
    window = args.window
    end_date = args.end_date
    result_dir = Path(args.result_dir)
    result_dir.mkdir(exist_ok=True)

    NOW = datetime.now()
    today_date = NOW.date()
    today_time_str = NOW.strftime("%H-%M-%S")
    print('time:', today_time_str)

    data_df = get_tokens_history(tokens, api_key, end_date, window)
    file_name = result_dir / "result.csv"
    if file_name.is_file():
        file_name.unlink()

    data_df.to_csv(file_name, sep='\t', encoding='utf-8')
 

today = str(date.today())


if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Fetch data from Covalent")
    parser.add_argument("--chain",
                        default="mainnet",
                        choices=["mainnet", "rinkeby"],
                        help="visualize train and test data")
    parser.add_argument("--api_key",
                        default="ckey_INSERT_KEY_HERE",
                        help="your api key on Covalent")
    parser.add_argument("--tokens",
                        default= ['aave', 'bal', 'link', 'crv', 'dai', 'gno', 'mke', 'rai', 'grt', 'uni', 'yfi'],
                        help="list of tokens")
    parser.add_argument("--result_dir",
                        default="./result",
                        help="the result directory path")
    parser.add_argument("--end_date",
                        default=today,
                        help="end date of history data")
    parser.add_argument("--window",
                        default=365,
                        help="the length of hisotry data to fetch")

    args, unknown = parser.parse_known_args()
    main(args)


results = pd.read_csv('result/result.csv', sep='\t')
results = results.iloc[: , 1:]
results = results.sort_values(by='date', ascending = False)
results

time: 09-04-17
start date: 2021-04-03, end date: 2022-04-03, window: 365


 73%|███████▎  | 8/11 [00:01<00:00,  5.82it/s]

Exclude mke token, because of unavailability of its data!


100%|██████████| 11/11 [00:02<00:00,  4.91it/s]


,date,aave,aave_priceChange,bal,bal_priceChange,link,link_priceChange,crv,crv_priceChange,dai,...,gno,gno_priceChange,rai,rai_priceChange,grt,grt_priceChange,uni,uni_priceChange,yfi,yfi_priceChange
364,2022-04-02,239.92671,0.991849,15.739734,0.978164,17.647917,1.039151,2.941860,1.067326,1.004063,...,427.52417,0.997142,3.029310,1.016293,4.950912,1.016293,11.874783,1.043494,23562.908,1.020749
363,2022-04-01,241.89835,1.136231,16.091105,0.998957,16.983007,0.971422,2.756290,1.012718,0.985235,...,428.74966,0.984403,2.980745,0.958600,4.871540,0.967385,11.379826,0.975385,23083.930,0.998761
362,2022-03-31,212.89539,0.944303,16.107908,0.958907,17.482630,1.006424,2.721674,0.953410,1.031158,...,435.54280,0.924013,3.109477,1.020384,5.035780,0.991658,11.667008,0.985259,23112.578,0.950196
361,2022-03-30,225.45238,1.050498,16.798204,1.166376,17.371038,1.036152,2.854675,1.025741,1.006136,...,471.36017,0.997866,3.047359,1.017009,5.078141,1.009882,11.841560,1.065211,24324.012,1.052945
360,2022-03-29,214.61480,1.277891,14.402050,1.040870,16.764954,1.020173,2.783037,1.079838,0.989712,...,472.36807,1.040465,2.996394,1.002060,5.028452,1.005989,11.116638,1.016300,23100.932,1.039466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2021-04-07,361.72836,0.931800,57.702667,0.969949,32.305860,0.918759,2.978009,0.930084,1.029874,...,166.79291,0.958771,3.071689,0.985482,3.751978,1.050640,29.568062,0.932658,41790.566,1.081128
3,2021-04-06,388.20395,0.989451,59.490387,1.004000,35.162487,1.095519,3.201870,0.979824,1.013867,...,173.96533,1.015533,3.116941,1.022648,3.571136,1.018253,31.702991,1.039682,38654.594,1.023620
2,2021-04-05,392.34274,0.982603,59.253372,1.046750,32.096660,1.046868,3.267800,1.093549,0.991054,...,171.30447,1.025083,3.047913,1.013111,3.507121,1.380267,30.492973,1.023405,37762.640,1.054807
1,2021-04-04,399.28930,1.017995,56.607018,0.924156,30.659700,1.010924,2.988254,0.990154,0.982580,...,167.11281,0.963648,3.008469,0.971235,2.540900,0.950977,29.795600,0.978728,35800.516,0.910378
